In [ ]:
Mount your google drive & symlink it to /drive

SyntaxError: ignored

# SSH Tunnel Setup .
In order to run *ssh* into the Google Colab Virtual Machine (VM), we need to: 
*   Start the ssh daemon on the VM
*   Reverse proxy **ngrok** 






## Installation 
Install the python wrapper to interface with ngrok.<br>
The colab-ssh is a light-weight library that enables you to connect to a Google Colab VM using an SSH tunnel.<p>

**Note**: colab-ssh is easiest way to install the ssh daemon and run the 
daemon. However, for finer control other methods are prefered.

In [11]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

  Using cached https://files.pythonhosted.org/packages/5e/95/4936b75ebb5d6dcd915a5ebb56ad1341de7f448459633a3f07ec34f207bc/colab_ssh-0.2.64-py3-none-any.whl


## Setup

To setup the reverse proxy we need ngrok authentication token. The token can be obtained from https://dashboard.ngrok.com/auth/your-authtoken. You may have to register if you don't have an account.<p>

Note down the secret key that you input to use it while tunnelling through ssh.<br> 
Note down the Hostname, User, and Port for reverse proxy.  

In [12]:
from colab_ssh import launch_ssh, init_git
ngrokToken = '1k6o811Gdt5dhRFYTxzdJGNdWqM_6HRtaxQMH9cjwasr4uJni'

from getpass import getpass
secret = getpass('Enter the secret key: ')

launch_ssh(ngrokToken, secret)

Enter the secret key: ··········
Successfully running 2.tcp.ngrok.io:17781
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 2.tcp.ngrok.io
		User root
		Port 17781
	  


## Instruction to SSH from a Terminal

From a terminal run the following command


> ssh \<User>@\<HostName> -p \<Port> <br>
> Ex: *ssh root@2.tcp.ngrok.io -p 17781*

When prompted for the **password** enter the **secret key**.

In [ ]:
#! export PATH=/usr/local/nvidia/bin:/usr/local/cuda/bin:$PATH




# Make sure that GPU is attached.
Execute the following cell.<br>
When the GPU is not attached the message(output) for this cell will indicate that the "NVIDIA-SMI" failed. If the GPU is not attached, set the Hardware Accelerator to GPU from colab menu:<br>
> *Runtime -> Change runtime type -> Hardware Accelerator*.    

Re-run the cell if you happpened to set up the Hardware Accelerator to make sure the the GPU is attached. 

In [7]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Thu Nov 12 00:43:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!service sshd start
!ps -ef | grep "sshd"

sshd: unrecognized service
root         247      56  0 00:43 ?        00:00:00 /bin/bash -c ps -ef | grep "sshd"
root         249     247  0 00:43 ?        00:00:00 grep sshd


# Debugging

In [ ]:
!which nvidia-smi
!echo $PATH
#!service ssh stop
!which nvcc

/usr/bin/nvidia-smi
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
/usr/local/cuda/bin/nvcc


In [ ]:
!whoami

root


# Run CUDA from Notebook

Run the command to install a small extension to run nvcc from the Notebook cells.

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pvxcki_g
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pvxcki_g
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=fe791f85de40778123bad70a088a797aa16365261a67dd1a4c3ce8d8d106348a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zuqk0230/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


Load the extension using the code given below

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu 
#include <iostream> 

int main() 
{ 
 printf("Welcome to the CUDA world!");
 return 0;
} 


Welcome to the CUDA world!


# FAQ

In [ ]:
! export PATH=/usr/local/nvidia/bin:/usr/local/cuda/bin:$PATH